# Filmproduktion Szenen Management Schedule 1

## Ziel des Notebooks
Dieses Notebook demonstriert, wie man eine Neo4j-Datenbank verwendet, um Filmproduktionsszenen zu verwalten, inklusive Erstellung von Szenen, Relationen und Gewichtungen basierend auf gemeinsamen Ressourcen. Außerdem wird ein Minimal Spanning Tree (MST) generiert, um die Planung von Dreharbeiten zu optimieren.


### Verbindung zu Neo4j Datenbank aufbauen

In [14]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

## Erstellung von Knoten und Kanten

In [18]:
def add_scene(tx, scene_id, location, actors, equipment, duration):
    # Szene-Knoten mit eingebetteten Listen von Schauspielenden und Ausrüstungen sowie der Dauer erstellen
    create_scene_query = """
    MERGE (s:Scene {id: $scene_id})
    ON CREATE SET s.location = $location, s.actors = $actors, s.equipment = $equipment, s.duration = $duration
    """
    tx.run(create_scene_query, scene_id=scene_id, location=location, actors=actors, equipment=equipment, duration=duration)

def create_edges(tx):
    # Kanten zwischen Szenen erstellen, die gemeinsame Schauspieler, Ausrüstungen oder Zeitpläne haben
    create_edge_query = """
    MATCH (s1:Scene), (s2:Scene)
    WHERE s1.id < s2.id AND
          (ANY(actor IN s1.actors WHERE actor IN s2.actors) OR
           ANY(equipment IN s1.equipment WHERE equipment IN s2.equipment) )           
    MERGE (s1)-[:RELATED]-(s2) 
    """
    # edges ungerichtet
    tx.run(create_edge_query)



## Gewichte setzen von Kanten
Niedrieges Gewicht für stärkere Verbindung -> höhere Priorität für das nacheinenader drehen.

Gewichtung niedriger wenn:
- Anzahl geinsames Equipment
- Anzahl gemeisame Actor

In [5]:
def set_weights(tx):
    set_weights_query = """
    MATCH (n:Scene)-[r:RELATED]->(m:Scene)
    WITH n, m, r,
         [x IN n.actors WHERE x IN m.actors] AS sharedActors,
         [y IN n.equipment WHERE y IN m.equipment] AS sharedEquipment
    SET r.weight = 10 - (size(sharedActors) + size(sharedEquipment))
    RETURN n, r, m
    """
    tx.run(set_weights_query)

## Graphenprojektion 
Erstellung eines projizierten Graphen 

In [24]:
def create_graph_project(tx):
    create_graph_query = """
    CALL gds.graph.project(
      'graphSchedulePlan',
      'Scene',
      {
        RELATED: {
          properties: 'weight',
          orientation: 'UNDIRECTED'
        }
      }
    )
    """
    tx.run(create_graph_query)

## Ausführung Spanning Tree

In [25]:
def spanning_tree(tx, start_node_id):
    spanning_tree_query = """
    MATCH (n:Scene {id: $start_node_id})
    CALL gds.spanningTree.write('graphSchedulePlan', {
      sourceNode: n,
      relationshipWeightProperty: 'weight',
      writeProperty: 'writeWeight',
      writeRelationshipType: 'MINST'
    })
    YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    """
    tx.run(spanning_tree_query, start_node_id=start_node_id)

## MST Anzeigen

In [26]:
def display_tree(tx, start_node_id):
    tree_edges = []
    display_tree_query = """
    MATCH path = (n:Scene {id: $start_node_id})-[:MINST*]-()
    WITH relationships(path) AS rels
    UNWIND rels AS rel
    WITH DISTINCT rel AS rel
    RETURN startNode(rel).id AS Source, endNode(rel).id AS Destination, rel.writeWeight AS Weight
    """
    results = tx.run(display_tree_query, start_node_id=start_node_id)
    for record in results:
        tree_edges.append((record['Source'], record['Destination'], record['Weight']))
        print(record)
    return tree_edges

## Depth-First Search (DFS) zur Traversierung des Spanning Trees
um tief in jeden Branch des Baumes einzutauchen, bevor zu einem anderen Zweig zurückgekehrt wird. Dies kann dazu beitragen, eine tiefgehende Analyse der Verbindungen zwischen den Szenen durchzuführen.

In [37]:
def dfs_traversal(tx, start_scene_id):
    dfs_query = """
    MATCH (start:Scene {id: $start_scene_id})
    CALL gds.dfs.stream('graphSchedulePlan', {
        sourceNode: id(start)
    })
    YIELD path
    UNWIND nodes(path) AS node
    RETURN collect(node.id) AS visitOrder
    """
    result = tx.run(dfs_query, start_scene_id=start_scene_id)
    visit_order = []
    for record in result:
        visit_order.extend(record['visitOrder'])
    print("DFS Visit Order:", visit_order)
    return visit_order

## Breadth-First Search (BFS) zur Traversierung des Spanning Trees

um die Breite des Baumes zu erforschen, bevor tiefer in die nächste Ebene eingetaucht wird. Diese Methode kann dazu beitragen, eine sequentielle Drehreihenfolge basierend auf der Nähe der Szenen zu etablieren.

In [36]:
def bfs_traversal(tx, start_scene_id):
    bfs_query = """
    MATCH (start:Scene {id: $start_scene_id})
    CALL gds.bfs.stream('graphSchedulePlan', {
        sourceNode: id(start)
    })
    YIELD path
    UNWIND nodes(path) AS node
    RETURN collect(node.id) AS visitOrder
    """
    result = tx.run(bfs_query, start_scene_id=start_scene_id)
    visit_order = []
    for record in result:
        visit_order.extend(record['visitOrder'])
    print("BFS Visit Order:", visit_order)
    return visit_order

## Funktionen ausführen

In [39]:
def load_json_data(file_path, driver):
    with open(file_path, 'r') as file:
        data = json.load(file)
        scenes = data['scenes']
        scene_number = 1
        with driver.session() as session:
            for scene in scenes:
                session.write_transaction(add_scene, f"Scene {scene_number}", scene['location'], scene['actors'], scene['equipment'], scene['duration'])
                scene_number += 1
            session.write_transaction(create_edges)
            session.write_transaction(set_weights)
            session.write_transaction(create_graph_project)
            session.write_transaction(spanning_tree, 'Scene 4')
            session.write_transaction(display_tree, 'Scene 4')
            dfs_order = session.write_transaction(dfs_traversal, 'Scene 4')
            bfs_order = session.write_transaction(bfs_traversal, 'Scene 4')

# Pfad zur JSON-Datei
json_file_path = 'data/scenes.json'
load_json_data(json_file_path, driver)

# Ressourcen freigeben
driver.close()


C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_scene, f"Scene {scene_number}", scene['location'], scene['actors'], scene['equipment'], scene['duration'])


<Record Source='Scene 4' Destination='Scene 3' Weight=9.0>
<Record Source='Scene 3' Destination='Scene 2' Weight=8.0>
<Record Source='Scene 2' Destination='Scene 1' Weight=8.0>
<Record Source='Scene 1' Destination='Scene 5' Weight=9.0>
DFS Visit Order: ['Scene 4', 'Scene 3', 'Scene 2', 'Scene 1', 'Scene 5']
BFS Visit Order: ['Scene 4', 'Scene 3', 'Scene 1', 'Scene 2', 'Scene 5']


C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_edges)
C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(set_weights)
C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(spanning_tree, 'Scene 4')
C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(display_tree, 'Scene 4')
C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\784889139.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  dfs_order = session.write_transaction(dfs_traversal, 'Scene 4')
C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404

## Parallel drehbare Szenen identifizieren

In [41]:
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

def fetch_isolated_scenes():
    isolated_scenes = []

    with driver.session() as session:
        # Abfrage zum Finden von Szenen ohne ausgehende RELATED-Beziehungen
        result = session.run("""
            MATCH (s:Scene)
            WHERE NOT (s)-[:RELATED]->(:Scene)
            RETURN s.id AS id
        """)
        
        # Sammeln aller Szenenstandorte, die keine Abhängigkeiten haben
        isolated_scenes = [record['id'] for record in result]

    return isolated_scenes

def plan_scenes():
    isolated = fetch_isolated_scenes()
    print("Isolierte Szenen, die parallel gedreht werden könnten:")
    for scene in isolated:
        print(scene)

plan_scenes()

# Ressourcen freigeben
driver.close()


Isolierte Szenen, die parallel gedreht werden könnten:
Scene 4
Scene 5


## Schedule Plan zusammensetzen

In [43]:
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

def dfs_traversal(tx, start_scene_id):
    dfs_query = """
    MATCH (start:Scene {id: $start_scene_id})
    CALL gds.dfs.stream('graphSchedulePlan', {
        sourceNode: id(start)
    })
    YIELD path
    UNWIND nodes(path) AS node
    RETURN collect(node.id) AS visitOrder
    """
    result = tx.run(dfs_query, start_scene_id=start_scene_id)
    visit_order = []
    for record in result:
        visit_order.extend(record['visitOrder'])
    print("DFS Visit Order:", visit_order)
    return visit_order

def fetch_isolated_scenes():
    isolated_scenes = []
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            WHERE NOT (s)-[:RELATED]->(:Scene)
            RETURN s.id AS id
        """)
        isolated_scenes = [record['id'] for record in result]
    return isolated_scenes

def fetch_scene_details():
    scene_details = {}
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            RETURN s.id AS id, s.actors AS actors, s.equipment AS equipment, s.location AS location
        """)
        for record in result:
            scene_details[record['id']] = {
                'actors': record['actors'],
                'equipment': record['equipment'],
                'location': record['location']
            }
    return scene_details

def plan_scenes():
    isolated = fetch_isolated_scenes()
    scene_details = fetch_scene_details()
    
    # DFS Traversal order
    with driver.session() as session:
        dfs_order = session.write_transaction(dfs_traversal, 'Scene 4')

    # Erstellen eines Plans ohne doppelte Destinations
    seen = set()
    plan = []

    for scene in dfs_order:
        if scene not in seen:
            plan.append(scene)
            seen.add(scene)

    # Integrieren der isolierten Szenen
    new_plan = []
    inserted_isolated = False

    for scene in plan:
        if scene in isolated and not inserted_isolated:
            new_plan.append((" and ".join(isolated), "parallel drehbar"))
            inserted_isolated = True
        elif scene not in isolated:
            new_plan.append(scene)

    print("Schedule Plan:")
    for item in new_plan:
        if isinstance(item, tuple):
            scenes = item[0].split(" and ")
            details = [f"{scene} (Actors: {', '.join(scene_details[scene]['actors'])}, Equipment: {', '.join(scene_details[scene]['equipment'])}, Location: {scene_details[scene]['location']})" for scene in scenes]
            print(", ".join(details) + " " + item[1])
        else:
            details = scene_details[item]
            print(f"{item} (Actors: {', '.join(details['actors'])}, Equipment: {', '.join(details['equipment'])}, Location: {details['location']})")

# Ausführen der Planungsfunktion
plan_scenes()

# Ressourcen freigeben
driver.close()


DFS Visit Order: ['Scene 4', 'Scene 3', 'Scene 2', 'Scene 1', 'Scene 5']
Schedule Plan:
Scene 4 (Actors: Nina Kraft, Oscar Wilde, Equipment: Camera W, Gimbal D, Camera Z, Location: Bahnhofstrasse), Scene 5 (Actors: Sara Löwe, Felix Sonne, Equipment: Camera V, Microphone A, Location: ETH Zürich) parallel drehbar
Scene 3 (Actors: Tom Braun, Johanna Schmidt, Lisa Feld, Equipment: Camera Z, Boom Mic B, Location: Bellevue)
Scene 2 (Actors: Max Müller, Lisa Feld, Equipment: Drone Y, Camera X, Boom Mic B, Location: Uetliberg)
Scene 1 (Actors: Max Müller, Johanna Schmidt, Equipment: Camera X, Microphone A, Location: Zürich Zentrum)


C:\Users\Noemi\AppData\Local\Temp\ipykernel_22404\2044792590.py:58: DeprecationWarning: write_transaction has been renamed to execute_write
  dfs_order = session.write_transaction(dfs_traversal, 'Scene 4')
